# Install libraries and import them

In [4]:
!pip install openai python-dotenv tqdm requests beautifulsoup4
!pip install --upgrade pinecone


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import minsearch
import json
from openai import OpenAI  # OpenAI API client
import pinecone  # Pinecone client library
# print(pinecone.__version__)  # Print Pinecone version (optional)
from pinecone import Pinecone, ServerlessSpec  # For setup and serverless config
import hashlib  # For hashing (e.g., file IDs)
import os  # OS operations and env vars
from datetime import datetime  # Date and time handling
from tqdm import tqdm  # Progress bars (e.g., looping through files)
import requests  # HTTP requests (e.g., for web scraping or API calls)
import re  # Regular expressions (e.g., for pattern matching in strings)
from bs4 import BeautifulSoup  # HTML parsing
from dotenv import load_dotenv  # Load .env file

In [6]:
load_dotenv()
# Now retrieve the keys
openai_key = os.getenv("OPENAI_API_KEY")
pinecone_key = os.getenv("PINECONE_API_KEY")

In [7]:
with open('rk.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [8]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        
        # Fix: normalize text fields to strings
        for field in ["question", "text", "section"]:
            if isinstance(doc.get(field, ""), list):
                doc[field] = " ".join(doc[field])
            elif doc.get(field) is None:
                doc[field] = ""
            else:
                doc[field] = str(doc[field])

        documents.append(doc)


In [9]:
documents[0]

{'section': 'Before Each Semester',
 'question': 'What should be done before the semester begins?',
 'text': 'Reserve the I7 CIP room (tell Stephanie). Check the expected number of students that will enroll. Plan the number of tutors and time slots based on expected student count. Contact students to join as tutors, starting with former tutors. Email previous year students based on RK grades and notify Erika for paperwork. Create StudOn content for the RK semester including Parallelgruppen and Vierergruppen. Use previous year examples as templates (e.g., SS24). Adjust Parallelgruppe registration via StudOn: Übungen → Übungsbereich → Anmeldung zu den Übungen. Edit the Zeitplan to reflect Feiertage and Vorlesungsfrei Tage. Schedule and email tutors about the first Zoom meeting, using DFN Terminplaner. Edit the RK_0_Org file in the lectures folder. Request StudOn access from I7admin for RK lectures. Create rkXYint, rkXY, rkXYabgabe email addresses (deprecated from RK25 onwards). Set corre

In [10]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [11]:
question = 'what to do before the semester?'

In [12]:
index.fit(documents)

In [ ]:
boost = {'question': 3.0, 'section': 0.5}
#shift+tab
results = index.search(
        query=question,
        filter_dict={'course': 'rechnerkommunikation-preparation-guide'},
        boost_dict=boost,
        num_results=5
    )


In [ ]:
results

# Use OpenAI

In [15]:
# Initialize the OpenAI client
client = OpenAI(
    api_key=openai_key
)

# Initialize Pinecone v3 client
pc = Pinecone(
    api_key=pinecone_key
)

In [17]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": question}]
)

response.choices[0].message.content

"Preparing for a new semester involves a combination of organization, goal-setting, and mental preparation to ensure you start off on the right foot. Here are some steps you can take:\n\n1. **Review Your Courses**: Look at the syllabus or course descriptions for your upcoming classes. Familiarize yourself with the topics you'll be studying and note any required materials or textbooks.\n\n2. **Get Organized**: Create a semester calendar with important dates, such as exams, assignment deadlines, and any other commitments. Set up a system for organizing your class notes, digital files, and other materials.\n\n3. **Purchase Materials**: Acquire any required textbooks, notebooks, and supplies. Consider whether you need any specific software or tools for certain courses.\n\n4. **Set Goals**: Define what you want to achieve academically and personally during the semester. Set specific, measurable, attainable, relevant, and time-bound (SMART) goals.\n\n5. **Establish a Routine**: Plan your dai